In [1]:
#coding=utf-8
import re
import jieba as jb
import os
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords
 
# 对句子进行分词
def seg_sentence(sentence):
    sentence = re.sub(u'[0-9\.]+', u'', sentence)
    '''jb.add_word('光线摄影学院')        # 这里是加入用户自定义的词来补充jieba词典。
    jb.add_word('曾兰老师')         # 同样，如果你想删除哪个特定的未登录词，就先把它加上然后放进停用词表里。
    jb.add_word('网页链接')
    jb.add_word('E%')
    jb.add_word('https')
    jb.add_word('微博视频')'''
    sentence_seged = jb.cut(sentence.strip())
    stopwords = stopwordslist('stopwords_cn_en.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords and word.__len__()>1:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr
 
def seg_file(input_file,output_file):#单个txt
    inputs = open(input_file, 'r', encoding='utf-8')
    outputs = open(output_file, 'w',encoding='utf-8')
    for line in inputs:
        line_seg = seg_sentence(line)  # 这里的返回值是字符串
        outputs.write(line_seg + '\n')
    outputs.close()
    inputs.close()
 
def seg_dir(input_dir,output_dir):#整个文件夹
    filelist=os.listdir(input_dir)
    for files in filelist:
        inputs = open(input_dir+'/'+files, 'r', encoding='utf-8')
        outputs = open(output_dir+'/'+files, 'w',encoding='utf-8')
        for line in inputs:
            line_seg = seg_sentence(line)  # 这里的返回值是字符串
            outputs.write(line_seg + '\n')
        outputs.close()
        inputs.close()
 
 
seg_dir("original_input","processed_input")
#seg_file("original_input/绍兴.txt","processed_input/绍兴.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ldeng\AppData\Local\Temp\jieba.cache
Loading model cost 0.894 seconds.
Prefix dict has been built successfully.


In [2]:
#coding=utf-8
import codecs
import os
from gensim import corpora
from gensim.models import LdaModel
from gensim.corpora import Dictionary
 
def classify(file):
    train = []
    fp = codecs.open('processed_input/'+file,'r',encoding='utf8')
    for line in fp:
        if line != '':
            line = line.split()
            train.append([w for w in line])
 
    dictionary = corpora.Dictionary(train)
 
    corpus = [dictionary.doc2bow(text) for text in train]
 
    lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=100)
    # num_topics：主题数目
    # passes：训练伦次
    # num_words：每个主题下输出的term的数目
 
    if os.path.exists('txt_output/'+file):
        os.remove('txt_output/'+file)
    f=open('txt_output/'+file,'w',encoding='utf-8')
    for topic in lda.print_topics(num_words = 10):
        termNumber = topic[0]
        print(topic[0], ':', sep='')
        f.write(str(topic[0])+':\n')#输出保存到文件
        listOfTerms = topic[1].split('+')
        for term in listOfTerms:
            listItems = term.split('*')
            result='  '+listItems[1]+'('+str(listItems[0])+')'
            print(result)
            f.write(result+'\n')#输出保存到文件
    f.close()
classify('xk.txt')

0:
  "中國" (0.015)
  "新疆" ( 0.011)
  "他們" ( 0.009)
  "独立" ( 0.008)
  "支持" ( 0.008)
  "美國" ( 0.006)
  "中共" ( 0.006)
  "BBC" ( 0.005)
  "一個" ( 0.005)
  "北爱尔兰"( 0.005)
1:
  "BBC" (0.033)
  "China" ( 0.013)
  "people" ( 0.010)
  "news" ( 0.008)
  "Chinese" ( 0.007)
  "Xinjiang" ( 0.007)
  "UK" ( 0.006)
  "fake" ( 0.005)
  "The" ( 0.005)
  "滤镜"( 0.005)
2:
  "新疆" (0.029)
  "中国" ( 0.021)
  "他们" ( 0.019)
  "教育" ( 0.013)
  "西方" ( 0.012)
  "恐怖分子" ( 0.012)
  "你们" ( 0.009)
  "BBC" ( 0.008)
  "支持" ( 0.008)
  "国家"( 0.007)
3:
  "我们" (0.029)
  "Ireland" ( 0.014)
  "Northern" ( 0.011)
  "freedom" ( 0.011)
  "Scotland" ( 0.010)
  "independence" ( 0.010)
  "When" ( 0.008)
  "人口" ( 0.003)
  "BBC" ( 0.003)
  "建议"( 0.003)
4:
  "你们" (0.036)
  "中国" ( 0.020)
  "英国" ( 0.019)
  "穆斯林" ( 0.018)
  "新疆" ( 0.016)
  "国家" ( 0.013)
  "自己" ( 0.012)
  "他们" ( 0.012)
  "BBC" ( 0.010)
  "我们"( 0.010)


In [3]:
#coding=utf-8
import codecs
import os
 
import gensim
import OpenSSL.crypto
import pyLDAvis.gensim_models
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models import LdaModel
 
 
def visualize(file):
    train = []
    fp = codecs.open('processed_input/'+file,'r',encoding='utf8')
    for line in fp:
        if line != '':
            line = line.split()
            train.append([w for w in line])
 
    dictionary = corpora.Dictionary(train)
 
    corpus = [dictionary.doc2bow(text) for text in train]
 
    lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=100)
    # num_topics：主题数目
    # passes：训练伦次
    # num_words：每个主题下输出的term的数目
 
    if os.path.exists('txt_output/'+file):
        os.remove('txt_output/'+file)
    f=open('txt_output/'+file,'w',encoding='utf-8')
    for topic in lda.print_topics(num_words = 10):
        termNumber = topic[0]
        print(topic[0], ':', sep='')
        f.write(str(topic[0])+':\n')#输出保存到文件
        listOfTerms = topic[1].split('+')
        for term in listOfTerms:
            listItems = term.split('*')
            result='  '+listItems[1]+'('+str(listItems[0])+')'
            print(result)
            f.write(result+'\n')#输出保存到文件
    f.close()
    #dictionary = gensim.corpora.Dictionary.load('lda.dict')
    #corpus = gensim.corpora.MmCorpus('lda.mm')
    #lda = models.ldamodel.LdaModel.load('lda.lda')
    vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
    pyLDAvis.save_html(vis, 'html_output/'+file[:-4]+'.html')
 
visualize("xk.txt")

D:\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
D:\Anaconda3\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5
D:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecate

0:
  "BBC" (0.025)
  "China" ( 0.014)
  "people" ( 0.011)
  "Chinese" ( 0.008)
  "Xinjiang" ( 0.007)
  "UK" ( 0.006)
  "The" ( 0.005)
  "terrorists" ( 0.005)
  "country" ( 0.004)
  "When"( 0.004)
1:
  "Ireland" (0.014)
  "Northern" ( 0.011)
  "news" ( 0.011)
  "Scotland" ( 0.010)
  "independence" ( 0.010)
  "wwwyoutubecom" ( 0.009)
  "watch" ( 0.009)
  "youtube" ( 0.008)
  "freedom" ( 0.008)
  "滤镜"( 0.007)
2:
  "新疆" (0.026)
  "中国" ( 0.011)
  "教育" ( 0.011)
  "BBC" ( 0.009)
  "他们" ( 0.008)
  "民族" ( 0.007)
  "宗教" ( 0.007)
  "共产党" ( 0.006)
  "我们" ( 0.006)
  "集中营"( 0.005)
3:
  "你们" (0.037)
  "中国" ( 0.026)
  "他们" ( 0.020)
  "我们" ( 0.018)
  "新疆" ( 0.018)
  "英国" ( 0.016)
  "国家" ( 0.015)
  "穆斯林" ( 0.014)
  "BBC" ( 0.012)
  "支持"( 0.011)
4:
  "新疆" (0.016)
  "中國" ( 0.014)
  "他們" ( 0.009)
  "BBC" ( 0.008)
  "中共" ( 0.006)
  "教育" ( 0.006)
  "美國" ( 0.006)
  "西方" ( 0.006)
  "一個" ( 0.005)
  "自己"( 0.005)


D:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
